# Planning Methods: Part II, Spring 2021

# Lab 1: Stats Review and Plotting

**About This Lab**
* We will be running through this notebook together. If you have a clarifying question or other question of broad interest, feel free to interrupt or use a pause to unmute and ask it! If you have a question that may result in a one-on-one breakout room (think: detailed inquiry, conceptual question, or help debugging), please ask it in the chat!
* We recognize learning Python via Zoom comes with its challenges and that there are many modes of learning. Please go with what works best for you. That might be printing out the Jupyter notebook, duplicating it such that you can refer to the original, working directly in it. Up to you! There isn't a single right way.
* This lab requires that you download the following file and place it in the same directory as this Jupyter notebook:
    * `property_data.csv`

## Objectives
By the end of this lab, you will have reviewed how to:
1. Read and write files
2. Check for and drop nulls
3. Export data
4. Create subdataframes
5. Produce descriptive statistics
6. Conduct statistical tests

You will also learn how to:
1. Check for outliers 
2. Define a function

## 1 Import packages

In [13]:
import pandas as pd, numpy as np, matplotlib as mpl, math

pd.options.display.float_format = '{:.2f}'.format
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

%matplotlib inline
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import t, chisquare, iqr
from scipy.stats import ttest_ind

import warnings 
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
pip install researchpy

In [ ]:
import researchpy as rp

In [ ]:
# filepath for csv == 'Lab Data/property_data.csv'

## 2 Connect to and clean data

In [6]:
raw = pd.read_csv('property_data.csv')
raw.head()

,id,price_000,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,apt,floor,...,pcnt_indu,pcnt_com,pcnt_insti,pcnt_vacant,pcn_green,homicides,house,ses_bin,lnprice,price_hi
0,40003,60000.00,1.00,0.00,0.00,0.00,0.00,0.00,1,5.00,...,0.00,0.00,0.49,0.00,1.74,39.92,0,0,11.00,0
1,40039,39000.00,1.00,0.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,9.21,1.73,0.00,0.69,46.00,1,0,10.57,0
2,40007,140000.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,15.41,1.32,0.00,0.54,46.00,1,0,11.85,1
3,40008,38000.00,0.00,1.00,0.00,0.00,0.00,0.00,1,1.00,...,0.00,8.16,5.57,0.00,1.55,45.87,0,0,10.55,0
4,40010,110000.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,8.11,5.53,0.00,1.58,45.88,1,0,11.61,1


In [7]:
raw.dtypes

id               int64
price_000      float64
yr_2001        float64
yr_2002        float64
yr_2003        float64
yr_2004        float64
yr_2005        float64
yr_2006        float64
apt              int64
floor          float64
age_0_10       float64
age_10_20      float64
age_20_more    float64
num_room       float64
num_bath       float64
num_garage     float64
area_m2        float64
ses            float64
pop_dens       float64
pcnt_indu      float64
pcnt_com       float64
pcnt_insti     float64
pcnt_vacant    float64
pcn_green      float64
homicides      float64
house            int64
ses_bin          int64
lnprice        float64
price_hi         int64
dtype: object

In [8]:
# convert id variable type
raw['id'] = raw['id'].astype(str)
raw.dtypes

id              object
price_000      float64
yr_2001        float64
yr_2002        float64
yr_2003        float64
yr_2004        float64
yr_2005        float64
yr_2006        float64
apt              int64
floor          float64
age_0_10       float64
age_10_20      float64
age_20_more    float64
num_room       float64
num_bath       float64
num_garage     float64
area_m2        float64
ses            float64
pop_dens       float64
pcnt_indu      float64
pcnt_com       float64
pcnt_insti     float64
pcnt_vacant    float64
pcn_green      float64
homicides      float64
house            int64
ses_bin          int64
lnprice        float64
price_hi         int64
dtype: object

### 2.1 Drop nulls

In [9]:
# check for null values
raw.isnull().sum()

id             0
price_000      2
yr_2001        2
yr_2002        2
yr_2003        2
yr_2004        2
yr_2005        2
yr_2006        2
apt            0
floor          1
age_0_10       1
age_10_20      1
age_20_more    1
num_room       1
num_bath       1
num_garage     1
area_m2        2
ses            4
pop_dens       2
pcnt_indu      1
pcnt_com       1
pcnt_insti     1
pcnt_vacant    1
pcn_green      0
homicides      0
house          0
ses_bin        0
lnprice        0
price_hi       0
dtype: int64

In [10]:
# how many observations would be dropped?
null_values = raw.isnull().sum(axis = 1)

num_obs = len(raw[null_values>0])

print(num_obs, len(raw))
raw[null_values>0]

9 3985


,id,price_000,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,apt,floor,...,pcnt_indu,pcnt_com,pcnt_insti,pcnt_vacant,pcn_green,homicides,house,ses_bin,lnprice,price_hi
1,40039,39000.00,1.00,0.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,9.21,1.73,0.00,0.69,46.00,1,0,10.57,0
12,40018,65000.00,0.00,0.00,0.00,0.00,0.00,0.00,1,3.00,...,0.00,8.14,7.62,0.00,0.57,46.00,0,0,11.08,0
50,40018,89191.50,1.00,0.00,0.00,0.00,0.00,0.00,1,3.00,...,nan,nan,nan,nan,0.14,41.06,0,0,11.40,0
63,40197,57500.00,nan,nan,nan,nan,nan,nan,1,1.00,...,0.00,0.00,1.49,0.00,0.21,35.76,0,0,10.96,0
107,40330,145000.00,1.00,0.00,0.00,0.00,0.00,0.00,0,nan,...,0.00,0.00,1.60,0.00,0.61,34.00,1,0,11.88,1
124,40495,40000.00,1.00,0.00,0.00,0.00,0.00,0.00,1,7.00,...,0.00,0.00,1.19,0.00,0.00,38.88,0,0,10.60,0
477,42486,nan,nan,nan,nan,nan,nan,nan,0,0.00,...,0.00,4.74,0.83,0.00,0.58,46.00,1,0,11.61,1
2523,506972,nan,1.00,0.00,0.00,0.00,0.00,0.00,1,2.00,...,0.44,3.67,17.18,5.52,5.41,137.38,0,0,11.08,0
3477,504693,52000.00,0.00,0.00,0.00,0.00,0.00,1.00,1,4.00,...,53.14,0.00,1.02,3.93,0.22,234.00,0,0,10.86,0


In [11]:
# drop NaN values
df = raw.dropna().reset_index(drop = True)
df.head()

,id,price_000,yr_2001,yr_2002,yr_2003,yr_2004,yr_2005,yr_2006,apt,floor,...,pcnt_indu,pcnt_com,pcnt_insti,pcnt_vacant,pcn_green,homicides,house,ses_bin,lnprice,price_hi
0,40003,60000.00,1.00,0.00,0.00,0.00,0.00,0.00,1,5.00,...,0.00,0.00,0.49,0.00,1.74,39.92,0,0,11.00,0
1,40007,140000.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,15.41,1.32,0.00,0.54,46.00,1,0,11.85,1
2,40008,38000.00,0.00,1.00,0.00,0.00,0.00,0.00,1,1.00,...,0.00,8.16,5.57,0.00,1.55,45.87,0,0,10.55,0
3,40010,110000.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,8.11,5.53,0.00,1.58,45.88,1,0,11.61,1
4,40011,120000.00,0.00,1.00,0.00,0.00,0.00,0.00,0,0.00,...,0.00,0.00,0.00,0.00,0.40,46.00,1,0,11.70,1


### 2.2 Check for outliers
Plus a sneak preview of plots!

In [12]:
<img src = 'boxplots.jpg' width = 500>
> Source: https://towardsdatascience.com/understanding-boxplots-5e2df7bcbd51

SyntaxError: invalid syntax (<ipython-input-12-00003e6f73d9>, line 1)

In [ ]:
# visualize population density
x = df['pop_dens']
plt.boxplot(x)
plt.show()
plt.hist(x, 250)
plt.show()

In [ ]:
# visualize price

In [ ]:
# how many observations would be dropped if we got rid of 'price_000' outliers?
# step 1: calculate interquartile range

In [ ]:
# step 2: use the 1.5xIQR rule 

### 2.3 Export clean data (for future labs)
Name it whatever you'd like and remeber where you save it so you can access next week.

In [ ]:
df.to_csv('clean_property_data.csv', index = False)

### 2.4 Create sub-dataframe

In [ ]:
sub_df = df[['house','apt','price_000','age_0_10','age_20_more','pcn_green','num_room']].copy()
sub_df.head()

In [ ]:
# rename variables of interest
sub_df.rename(columns={"price_000":"price", 
                   "age_0_10":"age_new", 
                   "age_20_more":"age_old", 
                   "num_room":"rooms"}, inplace = True)

## 3 Describe variables

### 3.1 Continuous variable

In [ ]:
# descriptive stats for property price

In [ ]:
# if we're only interested in certain statistics, we can also call them up specifically 
# print the median and interquartile range

In [ ]:
# note the median is equal to the 50% percentile above and IQR is equal to the 75th percentile minus the 25th

Next week we'll learn how to use a histogram to visualize the distribution of a continous variable.


### 3.2 Discrete numeric variable (dummy variable)

In [ ]:
# descriptive stats for house dv

In [ ]:
# we can also use the value_counts function (in general, it gives us a better sense of categorical variables)

In [ ]:
# and we can normalize value_counts to get percentages

#### 3.2.1 Define a function

A function is a block of reusable code used to perform a single action that only runs when called. Python has many built in functions (like 'print') but you can also create your own.  

In [ ]:
# if we want to see counts and percentages together, we can concatenate these outputs into one table

# defining a function called 'tab' that can take in any dataframe and any variable and return the output below

def tab(df, x):
    """This function concatenates the counts and percentages for a single variable in a dataframe."""
    
    print ("Total Count", df[x].count())
    print ("Total Pct", sum(df[x].value_counts(normalize=True)))
    
    return pd.concat([ df[x].value_counts(), df[x].value_counts(normalize=True) ], 
                     axis=1, keys=('counts','pct'))

In [ ]:
# call the function
tab(sub_df, 'house')

In [ ]:
# what steps could you take to apply this function to a different dataframe? 
# (hint: df in the function, and df at the where we call it with 'house' aren't the same thing...)

#### 3.2.2 Stats for all variables

In [ ]:
# these functions have been helpful for individual variables, but say you want to see summary stats for ALL the 
# variables in your dataframe at once?

In [ ]:
# if you want to see summary statistics like this for just a few variables, see the appendix code at the bottom!

## 4 Statistical tests

In [ ]:
# define universal set of statistics to be called with ".agg" command

### 4.1 T-test (of means)

#### 4.1.1 Do apartments have different prices from houses?

In [ ]:
# descriptive price stats for apartment dv 
# groupby and aggregate functions are helpful for looking at crosstabulated summary statistics

In [ ]:
# create apt and non-apt price variables

In [ ]:
# run t-test

In [ ]:
# if you wanted to normalize the price of a property by the number of bedrooms, how would you change the code? 
# (hint: more than one right answer!!)

# create per room price variable in dataframe
sub_df['pp_rm'] = sub_df['price']/df['rooms']

# create variables for t-test
apt_rm_p = sub_df[df.apt == 1].pp_rm #Apartment Price per Room
n_apt_rm_p = sub_df[df.apt == 0].pp_rm #Non-Apartment Price per Room

# run t-test
ttest_ind(apt_rm_p, n_apt_rm_p, equal_var = False, nan_policy="omit")

In [ ]:
# can also run t-test with researchpy if that's your preference!

#### 4.1.2 Is the price of newer apartments different from older apartments?

In [ ]:
# what descriptive stats are relevant here? 
# create subdataframe, group price of apartments by new vs. old

In [ ]:
# create old and new apartment price variables

In [ ]:
# run t-test

### 4.2 Chi-square test (of proportions)

#### 4.2.1 Are houses more likely to be older (age_20_more) or younger?

In [ ]:
# descriptive stats (crosstab)

In [ ]:
# normalize by row ('index') - could also normalize by 'columns'

In [ ]:
# run chi-square test

In [ ]:
# view table

In [ ]:
# view results

### Appendix A - Defining Functions

In [ ]:
# showing null values in order
# will only show null counts for rows that have more than 0 null values

### define the function
def var_nulls (df):
    null_counts = df.isnull().sum() #sum of null counts attributed to a variable
    return null_counts[null_counts > 0].sort_values(ascending=False) #sort values greater than 0 largest to smallest

### call the function
var_nulls(raw)

# output shows blank list because we have no null values here

In [ ]:
# value counts - see counts and percentages together

### define the function
def tab(df, x):
    print ("Total Count", df[x].count())
    print ("Total Pct", sum(df[x].value_counts(normalize=True)))
    return pd.concat([df[x].value_counts(), df[x].value_counts(normalize=True)], 
                     axis=1, keys=('counts','pct'))

### call the function
tab(sub_df, 'house') #specify the dataframe and the variable in the parentheses

In [ ]:
# we used the groupby function before each statistical test we used - we can also call this up with one function!

### define the function
def grpby_stats (df, var1, var2):
    stat = ['count', 'mean', 'min', 'max', 'median', 'std'] #specify the statistics we want
    return df[var1].groupby(df[var2]).agg(stat) #group the first variable by the second variable, and aggregate stats

### call the function
grpby_stats(sub_df, 'price', 'apt') #specify the dataframe, key variabe, and grouping variable in the parentheses

# does this work for the test in 4.1.2, where we define apt_p = (sub_df[sub_df.apt == 1].price)? try it out!

In [ ]:
# if you want to see this for ALL the variables in your dataframe, you use this code.
# this asks Python to describe the entire dataframe, and transpose (T) the columns and rows
# try deleting the .T to see what happens if you don't use it - either way is fine!

sub_df.describe().T

In [ ]:
# what if you just want to view a few of your key variables of interest?

# list of key variables
key_var = ['apt','price','rooms']

# code calling a few variables 
sub_df[key_var].describe().T

# you could also use this code below if you don't want to separately define a list of variables: 
sub_df[['apt','price','rooms']].describe().T